# load

In [1]:
# read in the credentials as environment variables
import sys
import os
cwd = os.getcwd()
if cwd.__contains__('src'):
    env_vars = !cat /src/.env
elif cwd.startswith('/data/users/jqhu'):
    env_vars = !cat /data/users/jqhu/work/tamagotchi/smartFlies/.env
    sys.path.append('/data/users/jqhu/work/tamagotchi/smartFlies')
else:
    env_vars = !cat /gscratch/walkerlab/jqhu/smartFlies/tamagotchi/.env

for var in env_vars:
    key, value = var.split('=')
    os.environ[key] = value

from schemas.schema_v6 import *

2025-08-08 02:59:48.894865: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754621988.905898 1266954 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754621988.909292 1266954 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-08 02:59:48.922158: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[2025-08-08 02:59:50,084][INFO]: Connecting jq@dev0.uwcnc.net:3306
[2025-08-08 02:59:50,093][INFO]: Connected jq@dev0.uwcnc.n

In [2]:
# (TrainingConfig() & f'training_config_hash="074eef10d15f46153fe6433861b160b8"')
for f in schema.jobs.fetch():
    error = f[4].split(':')[0:2]
    if error[0] == 'ConnectionResetError':
        print(f[1]) # print hash
        
# (schema.jobs & 'key_hash="c89f5ef984bfb3e30b3db6ff6ae68fa6"').delete()

In [3]:
"""key_to_train = {}
for entry in schema.jobs.fetch():
    key_hash = entry[1]
    status = entry[4]
    train_hash = entry[3]['training_config_hash']
    key_to_train[key_hash] = [train_hash, status]
    
# find in the training config table
for key_hash, tpl in key_to_train.items():
    train_hash = tpl[0]
    status = tpl[1]
    training_config = TrainingConfig()
    print(f"key_hash: {key_hash}, train_hash: {train_hash}")
    training_config = (training_config & f'training_config_hash="{train_hash}"')
    
    print(training_config.fetch('save_dir'), "seed:", training_config.fetch('seed'), status)
"""

'key_to_train = {}\nfor entry in schema.jobs.fetch():\n    key_hash = entry[1]\n    status = entry[4]\n    train_hash = entry[3][\'training_config_hash\']\n    key_to_train[key_hash] = [train_hash, status]\n    \n# find in the training config table\nfor key_hash, tpl in key_to_train.items():\n    train_hash = tpl[0]\n    status = tpl[1]\n    training_config = TrainingConfig()\n    print(f"key_hash: {key_hash}, train_hash: {train_hash}")\n    training_config = (training_config & f\'training_config_hash="{train_hash}"\')\n    \n    print(training_config.fetch(\'save_dir\'), "seed:", training_config.fetch(\'seed\'), status)\n'

# schema jobs

In [ ]:
schema.jobs
# (schema.jobs & f'status="{"error"}"').delete()
# (schema.jobs & f'host="{"g3123"}"')

In [ ]:

train_hash  = (TrainingConfig & 'save_dir="/src/data/wind_sensing/apparent_wind_visual_feedback/loc_algo_linear_rot_slice_const_noisy_softreset_sw_dist_logstep_wind_0.01_train_std_pois_mag_narrow_wind/"').fetch('training_config_hash')
to_del = []
for j in schema.jobs.fetch():
    print(j[3]['training_config_hash'])
    if j[3]['training_config_hash'] not in train_hash:
        to_del.append(j[1])
        
# for h in to_del:
    # (schema.jobs & f'key_hash="{h}"').delete()


# get config

## from schema

In [ ]:
for item in schema.jobs.fetch():
    training_config_hash = item[3]['training_config_hash']
    # get from TrainingConfig
    config = (TrainingConfig() & f'training_config_hash="{training_config_hash}"').fetch()
    print(config['seed'] )

## from results

In [ ]:
for item in TrainingResult.fetch():
    print("seed",item[2])

In [ ]:
# len(hash_to_del)
# for h in hash_to_del:
#     (schema.jobs & f'key_hash="{h}"').delete()

##  which seeds are on which host

In [ ]:
hosts = (schema.jobs).fetch('host')
hosts = set(hosts)
hosts_of_interest = ['g3103', 'g3101', 'g3100', 'g3121']
seeds_of_interest = []
time_substr = '2025-07-07 05'

hash_to_del = []

for i, host in enumerate(hosts):
    if host not in hosts_of_interest:
        continue
    print(f"Host {i+1}: {host}")
    for job in (schema.jobs & f'host="{host}"').fetch():

        seeds = (TrainingConfig() & f'training_config_hash="{job[3]["training_config_hash"]}"').fetch('seed')
        if seeds:
            # print(f"  Job: {job[1]}")
            # print(f"  Training Config Hash: {job[3]['training_config_hash']}")
            # print(f"  Seeds: {seeds}")
            seeds_of_interest.extend(seeds)
        start_time = job[-1]
        start_time = start_time.strftime('%Y-%m-%d %H')
        if time_substr in start_time:
            print(f"  Start Time: {start_time}")
            print(f"  Seeds: {seeds}")
        else:
            hash_to_del.append(job[1])
            print(f"  Start Time: {start_time} - Not in time substring {time_substr}, deleting job with key_hash {job[1]}")

len(seeds_of_interest)


In [ ]:
# (schema.jobs & 'status="error"' & 'host LIKE "g%"') # find all jobs with error status on hosts starting with 'g'
# (schema.jobs & 'status="error"' & 'host LIKE "g%"').delete() # find all jobs with error status on hosts starting with 'g'
# (schema.jobs & 'status="error"' ).delete() # delete all jobs with error status
# (schema.jobs & 'host="a0d143aaaba8"')
# (schema.jobs & 'host="78579dfbf30a"')
# (schema.jobs & 'status="error"' & 'host="78579dfbf30a"').delete() # find all jobs with error status on hosts starting with 'g'

# insert into `TrainingResult` table

In [ ]:
for job in schema.jobs.fetch():
    training_config_hash = job[3]['training_config_hash']
    key_hash = job[1]
    hours_elapsed = 0
    entry = (TrainingConfig() & f'training_config_hash="{training_config_hash}"').fetch()
    seed = entry['seed'][0] if entry['seed'] else None
    exp = os.path.basename(os.path.dirname(str(entry['save_dir'])))
    print(f"Job hash: {key_hash}, Seed: {seed}, Experiment name: {exp}")
    
    # TrainingResult().insert1(
    #     dict(
    #         training_config_hash=training_config_hash,
    #         seed=seed,
    #         hours_elapsed=hours_elapsed,
    #     ),
    #     allow_direct_insert=True
    # )

# Training config

In [ ]:
TrainingConfig()
# (TrainingConfig() & 'seed="1477"')
# TrainingConfig().fetch('save_dir')

# for hash in ['c97448325b629989b2d82e8f2f325c55', '45513dd8ac9d9cdbb3a34f957436f7af', 'dee781bba1737b60f0e9b494f6acfdfc']:
#     print((TrainingConfig() & f'training_config_hash="{hash}"').fetch()[['save_dir', 'seed']])
# (TrainingConfig() & 'training_config_hash="d1cc1b54c140f764025f8fc169a34456"').fetch()[['save_dir', 'seed']]

# wait to delete this too loc_algo_linear_rot_onPlume_const_diff_min_noisy_sw_dist_logstep_wind_0.01_train_std_pois_mag_narrow_wind
# (TrainingConfig() & 'save_dir="/src/data/wind_sensing/apparent_wind_visual_feedback/haltere_sacd_loc_algo_rotCL_oot_const_300updates_rnn64/"')
# set(TrainingConfig().fetch('save_dir'))

# any seed not run yet?

In [ ]:
subset_config = (TrainingConfig() & 'save_dir="/src/data/wind_sensing/apparent_wind_visual_feedback/finetune_oob_resetPolicy_onPlume_cosine_loc_algo_sw_dist_logstep_wind_0.1_train_std_pois_mag_narrow_wind/"').fetch()
print(len(subset_config))

finished_seeds = TrainingResult().fetch('seed')
queued_runs = (schema.jobs).fetch()
queued_config_hash = [i[3]['training_config_hash'] for i in queued_runs]
queued_seeds = (TrainingConfig & [{'training_config_hash': hash_val} for hash_val in queued_config_hash]).fetch('seed')

subset_seeds = [i['seed'] for i in subset_config]

# find seeds that are in subset_config but not in finished_seeds or queued_seeds
missing_seeds = set(subset_seeds) - set(finished_seeds) - set(queued_seeds)
print("Missing seeds:", missing_seeds)

In [ ]:
# # filter by list values
# config_hash = []
# configs = (TrainingConfig() & 'save_dir="/src/data/wind_sensing/apparent_wind_visual_feedback/loc_algo_linear_rot_slice_noisy_only_sw_dist_logstep_wind_0.01_train_std_pois_mag_narrow_wind/"').fetch(as_dict=True)
# for conf in configs:
#     if conf['dataset'] == ['constantx5b5', 'poisson_noisy3x5b5', 'poisson_mag_narrow_noisy3x5b5']:
#         print(conf['training_config_hash'])
#         config_hash.append(conf['training_config_hash'])

In [ ]:
# # delete the training config with a specific hash
# for hash in config_hash:
#     (TrainingConfig() & f'training_config_hash="{hash}"').delete()

# Training Result

In [ ]:
# TrainingResult.populate(reserve_jobs=True)
# 8171898c23c5a659f66124c1e971b1b4
# 7d7cfda61322daa8d05d557dbc8fa2c7
# 8a14ed9755f248b49c29679f7ac785fa
# 
# TrainingResult().insert(
    
#     [dict(training_config_hash='8a14ed9755f248b49c29679f7ac785fa',
#         seed=15717,
#         hours_elapsed=0)]
#     , allow_direct_insert=True
# )

TrainingResult()
# TrainingResult() & 'training_config_hash="3f15cb3d1c75ac2c186262cef6af8baa"'
# TrainingResult() & 'seed="6409"'

In [ ]:
# training_config_hash = [item[0] for item in TrainingResult().fetch()]
# training_config = [item[0] for item in TrainingConfig().fetch()]
# to_be_done_exp = set(training_config).difference(set(training_config_hash))
# print(len(to_be_done_exp))
# undone = ['/plume_10057_5e0b837fa56b6a4dffff1a034adcff23.pt',
# '/plume_11357_85f2b252e3fb3537cc8b97e7299543d1.pt',
# '/plume_13356_edf288d268883f38671e8d8b3c6faa80.pt',
# '/plume_19417_4ae9997221924be5c7af5b45ebc2f399.pt',
# '/plume_23197_87f2d20ceaa4d4567390aef42c9a18b3.pt',
# '/plume_23311_010d39e8723185121880b9cb06092819.pt',
# '/plume_31287_f381b0d5c73c6a567dc6a53437118d1b.pt',
# '/plume_6337_48886f495fca96cc3bd4e5fae0eff333.pt',
# "/plume_8898_d8e5f14b5b73bcc1d45585b57d7f6b8c.pt"]

# undone = [exp.split('_')[-1].replace('.pt', '') for exp in undone]
# print(len(undone))
# print([exp for exp in to_be_done_exp if exp in undone])
# to_be_done_exp
# for hash in to_be_done_exp:
#     print((TrainingConfig() & f'training_config_hash="{hash}"').fetch()['seed'])
# # (TrainingConfig() & 'training_config_hash="061f81f6e646a49a94eb2d27c0bf41ef"')

# insert a hyak-based example 061824

# fintune

In [ ]:
dict_to_insert = {
 'save_dir': '/src/data/wind_sensing/apparent_wind_visual_feedback/finetune_oot_resetPolicy_onPlume_cosine_loc_algo_sw_dist_logstep_wind_0.1_train_std_pois_mag_narrow_wind/',
#  'seed': 2894, # to be sampled when inserting
 'outsuffix': '', # to be built by insert 
 'num_processes': 4, # nproc=8 for 4 concurrently agents. 
 'dataset': ['poisson_noisy3x5b5', 'poisson_mag_narrow_noisy3x5b5'],
 'num_env_steps': 10000000,
 'birthx': 0.1, # TODO: verify this is what Toha has
 'qvar': [0.5, 0.5], # Variance of init. location; higher = more off-plume initializations. Note this is set to 0 in evalCli 
 'diff_max': [0.8, 0.8],
 'diff_min': [0.2, 0.2],
 'birthx_linear_tc_steps': 0, 
 'apparent_wind': True,
 'apparent_wind_allo': False,
 'visual_feedback': True,
 'birthx_max': 1.0,
 'rnn_type': 'VRNN',
 'hidden_size': 64,
 'env_dt': 0.04,
 'dryrun': False,
 'algo': 'ppo',
 'lr': 0.0003,
 'eps': 1e-05,
 'alpha': 0.99,
 'gamma': 0.99,
 'use_gae': True,
 'gae_lambda': 0.95,
 'entropy_coef': 0.005,
 'value_loss_coef': 0.5,
 'max_grad_norm': 0.5,
 'cuda_deterministic': False,
 'num_steps': 2048,
 'ppo_epoch': 10,
 'num_mini_batch': 4,
 'clip_param': 0.2,
 'log_interval': 1,
 'save_interval': 100,
 'no_cuda': False,
 'use_proper_time_limits': False,
 'recurrent_policy': True,
 'use_linear_lr_decay': True,
 'env_name': 'plume',
 'log_dir': '/src/data/wind_sensing/apparent_wind_visual_feedback/logs/',
 'dynamic': False,
 'eval_type': 'skip',
 'eval_episodes': 20,
 'eval_interval': None,
 'weight_decay': 0.0001,
 'betadist': False,
 'stacking': 0,
 'masking': None,
 'stride': 1,
 'curriculum': False,
 'turnx': 1.0,
 'movex': 1.0,
 'auto_movex': False,
 'auto_reward': False,
 'loc_algo': 'on_plume_linear',
 'time_algo': 'uniform',
 'walking': False,
 'radiusx': 1.0,
 'diffusion_min': 1.0,
 'diffusion_max': 1.0,
 'r_shaping': ['step', 'cosine', 'finetune', 'reset_actor', 'reset_critic'],
 'wind_rel': True,
 'action_feedback': False,
 'squash_action': True,
 'flipping': False,
 'odor_scaling': True,
 'stray_max': 2.0,
 'test_episodes': 50,
 'viz_episodes': 10,
 'model_fname': '',
 'obs_noise': 0.0, # 16deg = 0.28 rad
 'act_noise': 0.0,
 'cuda': True,
 'if_vec_norm': 1,
 'if_train_actor_std': True,
 'rotate_by': 1, # True: rotate the wind direction and plume line randomly for generalization
 }

In [ ]:
for item in [[9712, "9712_334ca23297234746a0282ddd6bc6e381"],
[25349, "25349_3cef330900a57fc3f9a78e19aadc452b"],
[31727, "31727_6f88b79eb3b03ed34ae19e9c49f92595"],
[27905, "27905_ac2d85c83f6dae93444e435b3129cad2"],
[5562, "5562_771171cd71fb126cc07f3b0148a5c9a4"],
[4215, "4215_e08dd86a6aa8d34eb8bbdfd7169f90c4"],
[3532, "3532_dc82c1828c3f948e5147c6194d791350"],
[3929, "3929_2f388a0a7560b62a46440c7b78b238b2"],
[5959, "5959_428498a9f07478f9b1d6e9c284b997b0"],
[28594, "28594_3c8553f83409657c8954329d30ad1471"],
[1686, "1686_de9ad88ffd7e6295518474334a4a1326"],
[6013, "6013_9c56357ea1392dd8556eb91b2d506f94"]]:
    seed = item[0]
    hash = item[1]
    dict_to_insert['seed'] = seed
    dict_to_insert['outsuffix'] = hash
    TrainingConfig.insert1(dict_to_insert)

# new entry

In [ ]:
dict_to_insert = {
 'save_dir': '/src/data/wind_sensing/apparent_wind_visual_feedback/loc_algo_linear_rot_oob_onPlume_noisy_sw_dist_logstep_wind_0.1_train_std_pois_mag_narrow_wind/',
#  'outsuffix': '', # to be built by insert 
 'num_processes': 4, # nproc=8 for 4 concurrently agents. 
#  'seed': 2894, # to be sampled when inserting
 'dataset': ['poisson_noisy3x5b5', 'poisson_mag_narrow_noisy3x5b5'],
 'num_env_steps': 10000000,
 'birthx': 0.1, # TODO: verify this is what Toha has
 'qvar': [0.5, 0.5], # Variance of init. location; higher = more off-plume initializations. Note this is set to 0 in evalCli 
 'diff_max': [0.8, 0.8],
 'diff_min': [0.25, 0.25],
 'apparent_wind': True,
 'apparent_wind_allo': False,
 'visual_feedback': True,
 'birthx_linear_tc_steps': 6, 
 'birthx_max': 1.0,
 'rnn_type': 'VRNN',
 'hidden_size': 64,
 'env_dt': 0.04,
 'dryrun': False,
 'algo': 'ppo',
 'lr': 0.0003,
 'eps': 1e-05,
 'alpha': 0.99,
 'gamma': 0.99,
 'use_gae': True,
 'gae_lambda': 0.95,
 'entropy_coef': 0.005,
 'value_loss_coef': 0.5,
 'max_grad_norm': 0.5,
 'cuda_deterministic': False,
 'num_steps': 2048,
 'ppo_epoch': 10,
 'num_mini_batch': 4,
 'clip_param': 0.2,
 'log_interval': 1,
 'save_interval': 100,
 'no_cuda': False,
 'use_proper_time_limits': False,
 'recurrent_policy': True,
 'use_linear_lr_decay': True,
 'env_name': 'plume',
 'log_dir': '/src/data/wind_sensing/apparent_wind_visual_feedback/logs/',
 'dynamic': False,
 'eval_type': 'skip',
 'eval_episodes': 20,
 'eval_interval': None,
 'weight_decay': 0.0001,
 'betadist': False,
 'stacking': 0,
 'masking': None,
 'stride': 1,
 'curriculum': False,
 'turnx': 1.0,
 'movex': 1.0,
 'auto_movex': False,
 'auto_reward': False,
 'loc_algo': 'on_plume_linear', # slice linear gradually increases diff_min
 'time_algo': 'uniform',
 'walking': False,
 'radiusx': 1.0,
 'diffusion_min': 1.0,
 'diffusion_max': 1.0,
 'r_shaping': ['step', 'oob'],
 'wind_rel': True,
 'action_feedback': False,
 'squash_action': True,
 'flipping': False,
 'odor_scaling': True,
 'stray_max': 2.0,
 'test_episodes': 50,
 'viz_episodes': 10,
 'model_fname': '',
 'obs_noise': 0.0, # 16deg = 0.28 rad
 'act_noise': 0.0,
 'cuda': True,
 'if_vec_norm': 1,
 'if_train_actor_std': True,
 'rotate_by': 1, # True: rotate the wind direction and plume line randomly for generalization
 }

In [ ]:
for i in range(15):
    TrainingConfig.insert1(dict_to_insert)

In [ ]:
TrainingConfig().fetch()['outsuffix']
len(TrainingConfig().fetch()['outsuffix'])


# short segment

In [31]:
# new entry
dict_to_insert = {
 'save_dir': '/src/data/wind_sensing/apparent_wind_visual_feedback/haltere_sacd_loc_algo_rotCL_oot_const_300updates_rnn128/',
#  'outsuffix': '', # to be built by insert 
 'num_processes': 4, # nproc=8 for 4 concurrently agents. 
#  'seed': 2894, # to be sampled when inserting
 'dataset': ['constantx5b5'],
#  'dataset': ['poisson_noisy3x5b5', 'poisson_mag_narrow_noisy3x5b5'],
#  'num_env_steps': 10000000,
 'num_env_steps': 2500000,
 'birthx': 0.1, # TODO: verify this is what Toha has
 'qvar': [1.5], # Variance of init. location; higher = more off-plume initializations. Note this is set to 0 in evalCli 
 'diff_max': [0.8],
 'diff_min': [0.5],
 'apparent_wind': True,
 'apparent_wind_allo': False,
 'visual_feedback': True,
 'birthx_linear_tc_steps': 6, 
 'birthx_max': 1.0,
 'rnn_type': 'VRNN',
 'hidden_size': 128,
 'env_dt': 0.04,
 'dryrun': False,
 'algo': 'ppo',
 'lr': 0.0003,
 'eps': 1e-05,
 'alpha': 0.99,
 'gamma': 0.99,
 'use_gae': True,
 'gae_lambda': 0.95,
 'entropy_coef': 0.005,
 'value_loss_coef': 0.5,
 'max_grad_norm': 0.5,
 'cuda_deterministic': False,
 'num_steps': 2048,
 'ppo_epoch': 10,
 'num_mini_batch': 4,
 'clip_param': 0.2,
 'log_interval': 1,
 'save_interval': 100,
 'no_cuda': False,
 'use_proper_time_limits': False,
 'recurrent_policy': True,
 'use_linear_lr_decay': True,
 'env_name': 'plume',
 'log_dir': '/src/data/wind_sensing/apparent_wind_visual_feedback/logs/',
 'dynamic': False,
 'eval_type': 'skip',
 'eval_episodes': 20,
 'eval_interval': None,
 'weight_decay': 0.0001,
 'betadist': False,
 'stacking': 0,
 'masking': None,
 'stride': 1,
 'curriculum': False,
 'turnx': 1.0,
 'movex': 1.0,
 'auto_movex': False,
 'auto_reward': False,
 'loc_algo': 'slice', # slice linear gradually increases diff_min # location algorithm !!!!!
 'time_algo': 'uniform',
 'walking': False,
 'radiusx': 1.0,
 'diffusion_min': 1.0,
 'diffusion_max': 1.0,
 'r_shaping': ['step', 'rotate_by', 'haltere', 'saccade'],
 'wind_rel': True,
 'action_feedback': False,
 'squash_action': True,
 'flipping': False,
 'odor_scaling': True,
 'stray_max': 2.0,
 'test_episodes': 50,
 'viz_episodes': 10,
 'model_fname': '',
 'obs_noise': 0.0, # 16deg = 0.28 rad
 'act_noise': 0.0,
 'cuda': True,
 'if_vec_norm': 1,
 'if_train_actor_std': True,
 'rotate_by': 1, # True: rotate the wind direction and plume line randomly for generalization
 }

print('exp name', dict_to_insert['save_dir'].split('/')[-2],
      '\nreward shaping', dict_to_insert['r_shaping'], '\n loc algo',  
      dict_to_insert['loc_algo'], '\n curriculum', 
      dict_to_insert['birthx_linear_tc_steps'], 
      '\n learning rate', dict_to_insert['lr'],
      '\n num env steps', dict_to_insert['num_env_steps'],
      '')

exp name haltere_sacd_loc_algo_rotCL_oot_const_300updates_rnn128 
reward shaping ['step', 'rotate_by', 'haltere', 'saccade'] 
 loc algo slice 
 curriculum 6 
 learning rate 0.0003 
 num env steps 2500000 


In [32]:
for i in range(18):
    TrainingConfig.insert1(dict_to_insert)

No seed provided. Generating a new one.
r_shaping: ['step', 'rotate_by', 'haltere', 'saccade']


No seed provided. Generating a new one.
r_shaping: ['step', 'rotate_by', 'haltere', 'saccade']
No seed provided. Generating a new one.
r_shaping: ['step', 'rotate_by', 'haltere', 'saccade']
No seed provided. Generating a new one.
r_shaping: ['step', 'rotate_by', 'haltere', 'saccade']
No seed provided. Generating a new one.
r_shaping: ['step', 'rotate_by', 'haltere', 'saccade']
No seed provided. Generating a new one.
r_shaping: ['step', 'rotate_by', 'haltere', 'saccade']
No seed provided. Generating a new one.
r_shaping: ['step', 'rotate_by', 'haltere', 'saccade']
No seed provided. Generating a new one.
r_shaping: ['step', 'rotate_by', 'haltere', 'saccade']
No seed provided. Generating a new one.
r_shaping: ['step', 'rotate_by', 'haltere', 'saccade']
No seed provided. Generating a new one.
r_shaping: ['step', 'rotate_by', 'haltere', 'saccade']
No seed provided. Generating a new one.
r_shaping: ['step', 'rotate_by', 'haltere', 'saccade']
No seed provided. Generating a new one.
r_shaping:

# staged training

In [ ]:
base_exp = 'loc_algo_rotCL_oob_onPlume_const_300updates'
base_exp = 'loc_algo_rotCL_oot_onPlume_const_300updates'
base_exp = 'haltere_sacd_loc_algo_rotCL_oob_on_plume_const_300updates_rnn128'


In [10]:
base_exp = 'haltere_sacd_loc_algo_rotCL_oot_on_plume_const_300updates_rnn128'
tbl = (TrainingConfig() & f'save_dir="/src/data/wind_sensing/apparent_wind_visual_feedback/{base_exp}/"')
tbl = (TrainingConfig() & f'save_dir="/src/data/wind_sensing/apparent_wind_visual_feedback/{base_exp}/"')

# Fetch as list of dictionaries
rows_as_dicts = tbl.fetch(as_dict=True)

# Filter: keep only rows where at least one element in r_shaping contains "staged_"
rows_as_dicts = [row for row in rows_as_dicts if not any("staged_" in s for s in row['r_shaping'])]

unique_r_shaping = set(tuple(x['r_shaping']) for x in rows_as_dicts)
unique_loc_algo = set(x['loc_algo'] for x in rows_as_dicts)
unique_birthx_steps = set(x['birthx_linear_tc_steps'] for x in rows_as_dicts)
unique_lr = set(x['lr'] for x in rows_as_dicts)
unique_num_env_steps = set(x['num_env_steps'] for x in rows_as_dicts)
unique_dataset = set(tuple(x['dataset']) for x in rows_as_dicts)
unique_ppo_epoch = set(x['ppo_epoch'] for x in rows_as_dicts)
unique_num_processes = set(x['num_processes'] for x in rows_as_dicts)
unique_num_mini_batch = set(x['num_mini_batch'] for x in rows_as_dicts)


print('N base experiments found', len(rows_as_dicts),
      '\n reward shaping', unique_r_shaping, '\n loc algo',
      unique_loc_algo, '\n curriculum',
      unique_birthx_steps,
      '\n learning rate', unique_lr,
      '\n num env steps', unique_num_env_steps,
      '\n dataset', unique_dataset,
      '')
print('ppo_epoch', unique_ppo_epoch,
      '\n num processes', unique_num_processes,
      '\n num mini batch', unique_num_mini_batch)
r_shaping = [item for tup in unique_r_shaping for item in tup]
r_shaping.append('staged_rotCL_oot_onPlume_poisMagN3_5MilTS_sampleEParams')
dataset = ['poisson_mag_narrow_noisy3x5b5']
# dataset = ['poisson_noisy3x5b5']
# dataset = ['poisson_noisy3x5b5', 'poisson_mag_narrow_noisy3x5b5']

# qvar = [0.5, 0.5]
# diff_max = [0.8, 0.8]
# diff_min = [0.5, 0.5]
num_env_steps = 5000000
qvar = [0.5]
diff_max = [0.8]
diff_min = [0.5]
num_processes = 8
num_mini_batch = 8
ppo_epoch = 5

# new config
print('\nNew configs')
print('\nr_shaping', r_shaping, 
      '\ndataset', dataset, 
      '\nnum_env_steps', num_env_steps,
      '\nloc_algo', unique_loc_algo,
      '\nunique_birthx_steps', unique_birthx_steps,)
print('ppo_epoch', ppo_epoch,
      '\nnum_processes', num_processes,
      '\nnum_mini_batch', num_mini_batch,)



N base experiments found 30 
 reward shaping {('step', 'rotate_by', 'haltere', 'saccade')} 
 loc algo {'on_plume'} 
 curriculum {6} 
 learning rate {0.0003} 
 num env steps {2500000} 
 dataset {('constantx5b5',)} 
ppo_epoch {10} 
 num processes {4} 
 num mini batch {4}

New configs

r_shaping ['step', 'rotate_by', 'haltere', 'saccade', 'staged_rotCL_oot_onPlume_poisMagN3_5MilTS_sampleEParams'] 
dataset ['poisson_mag_narrow_noisy3x5b5'] 
num_env_steps 5000000 
loc_algo {'on_plume'} 
unique_birthx_steps {6}
ppo_epoch 5 
num_processes 8 
num_mini_batch 8


In [11]:
# for each entry in rows_as_dicts, insert a new entry with modified values
for i, row in enumerate(rows_as_dicts):
    new_row = row.copy()
    new_row['r_shaping'] = r_shaping
    new_row['dataset'] = dataset
    new_row['qvar'] = qvar
    new_row['diff_max'] = diff_max
    new_row['diff_min'] = diff_min
    new_row['num_env_steps'] = num_env_steps
    new_row['num_processes'] = num_processes
    new_row['num_mini_batch'] = num_mini_batch
    new_row['ppo_epoch'] = ppo_epoch
    
    # Insert the modified row into the TrainingConfig table
    TrainingConfig.insert1(new_row)
print(f"Inserted {len(rows_as_dicts)} new entries with modified values.")


r_shaping: ['step', 'rotate_by', 'haltere', 'saccade', 'staged_rotCL_oot_onPlume_poisMagN3_5MilTS_sampleEParams']
r_shaping: ['step', 'rotate_by', 'haltere', 'saccade', 'staged_rotCL_oot_onPlume_poisMagN3_5MilTS_sampleEParams']
r_shaping: ['step', 'rotate_by', 'haltere', 'saccade', 'staged_rotCL_oot_onPlume_poisMagN3_5MilTS_sampleEParams']
r_shaping: ['step', 'rotate_by', 'haltere', 'saccade', 'staged_rotCL_oot_onPlume_poisMagN3_5MilTS_sampleEParams']
r_shaping: ['step', 'rotate_by', 'haltere', 'saccade', 'staged_rotCL_oot_onPlume_poisMagN3_5MilTS_sampleEParams']
r_shaping: ['step', 'rotate_by', 'haltere', 'saccade', 'staged_rotCL_oot_onPlume_poisMagN3_5MilTS_sampleEParams']
r_shaping: ['step', 'rotate_by', 'haltere', 'saccade', 'staged_rotCL_oot_onPlume_poisMagN3_5MilTS_sampleEParams']


r_shaping: ['step', 'rotate_by', 'haltere', 'saccade', 'staged_rotCL_oot_onPlume_poisMagN3_5MilTS_sampleEParams']
r_shaping: ['step', 'rotate_by', 'haltere', 'saccade', 'staged_rotCL_oot_onPlume_poisMagN3_5MilTS_sampleEParams']
r_shaping: ['step', 'rotate_by', 'haltere', 'saccade', 'staged_rotCL_oot_onPlume_poisMagN3_5MilTS_sampleEParams']
r_shaping: ['step', 'rotate_by', 'haltere', 'saccade', 'staged_rotCL_oot_onPlume_poisMagN3_5MilTS_sampleEParams']
r_shaping: ['step', 'rotate_by', 'haltere', 'saccade', 'staged_rotCL_oot_onPlume_poisMagN3_5MilTS_sampleEParams']
r_shaping: ['step', 'rotate_by', 'haltere', 'saccade', 'staged_rotCL_oot_onPlume_poisMagN3_5MilTS_sampleEParams']
r_shaping: ['step', 'rotate_by', 'haltere', 'saccade', 'staged_rotCL_oot_onPlume_poisMagN3_5MilTS_sampleEParams']
r_shaping: ['step', 'rotate_by', 'haltere', 'saccade', 'staged_rotCL_oot_onPlume_poisMagN3_5MilTS_sampleEParams']
r_shaping: ['step', 'rotate_by', 'haltere', 'saccade', 'staged_rotCL_oot_onPlume_poisMag

# scaping the table

In [ ]:
TrainingConfig.drop()